# Analisis de señales raw 
### ZEMG V1.0

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import pandas as pd
import plotly.io as pio



def cargar_datos_desde_archivos(archivos):
    data_frames = []

    for archivo in archivos:
        # Leer el archivo CSV y almacenar los datos en un DataFrame, omitiendo la primera fila
        df = pd.read_csv(archivo, skiprows=1, names=["Muestras", "Valores"])

        # Extraer el nombre del dedo de la ruta del archivo
        dedo_nombre = archivo.split("/")[1].split(".")[0]

        global ventana
        ventana=5
        df["MovMean"]=df["Valores"].rolling(ventana).mean()

        # Agregar el nombre del dedo como una columna
        df["Dedo"] = dedo_nombre

        data_frames.append(df)

    # Concatenar todos los DataFrames en uno solo
    data = pd.concat(data_frames, ignore_index=True)

    global datos_cargados
    datos_cargados = data


def graficar_datos():
    if datos_cargados is None:
        raise ValueError(
            "Los datos no han sido cargados. Llama a 'cargar_datos_desde_archivos' primero."
        )

    # Crear un gráfico de líneas interactivo utilizando Plotly Express
    fig = px.line(
        datos_cargados,
        template="plotly_dark",
        x="Muestras",
        y="Valores",
        color="Dedo",
        title="Gráfico de Varios Dedos",
    )
   

    # Mostrar el gráfico interactivo utilizando Plotly
    fig.show()
    pio.write_html(fig, 'nombre_del_archivo.html')

def graficar_datos_movMean():
    if datos_cargados is None:
        raise ValueError(
            "Los datos no han sido cargados. Llama a 'cargar_datos_desde_archivos' primero."
        )

    # Crear un gráfico de líneas interactivo utilizando Plotly Express
    fig = px.line(
        datos_cargados,
        template="plotly_dark",
        x="Muestras",
        y="MovMean",
        color="Dedo",
        title="Gráfico de Varios Dedos - Movmean = " + str(ventana),
    )
    

    # Mostrar el gráfico interactivo utilizando Plotly
    fig.show()


# Lista de nombres de archivos CSV que deseas graficar juntos
archivos_a_graficar = [
    'Records/Indice.csv', 'Records/Medio.csv', 'Records/Anular.csv', 'Records/Meñique.csv', 'Records/Pulgar.csv','Records/Puño.csv','Records/Puño Flexionado.csv'
]

# Cargar los datos desde los archivos una vez
cargar_datos_desde_archivos(archivos_a_graficar)



## Graficos de la señal cruda de cada dedo y de la movmean

In [ ]:
graficar_datos()
graficar_datos_movMean()

## Analisis frecuencial (FFT)

In [ ]:
muestras_anular = datos_cargados[
    (datos_cargados['Muestras'] >= 100) & 
    (datos_cargados['Muestras'] <= 1400) & 
    (datos_cargados['Dedo'] == 'Anular')
]['Valores']

# Obtén los valores de la columna 'Valores' en ese rango de muestras del dedo "Anular"


# Realiza la FFT de los valores en el rango seleccionado del dedo "Anular"
fft_valores_rango_anular = np.fft.fft(muestras_anular)

# Encuentra la magnitud de las amplitudes de la FFT del dedo "Anular"
magnitud_fft_anular = np.abs(fft_valores_rango_anular)

# Encuentra las frecuencias correspondientes para el dedo "Anular"
frecuencias_anular = np.fft.fftfreq(len(muestras_anular))

mitad_frecuencias_anular = frecuencias_anular[:len(frecuencias_anular)//2]
mitad_magnitudes_fft_anular = np.abs(fft_valores_rango_anular)[:len(magnitud_fft_anular)//2]

# Selecciona el rango de muestras de interés (450 a 550) del dedo "Índice"
muestras_rango_indice = datos_cargados[
    (datos_cargados['Muestras'] >= 100) & 
    (datos_cargados['Muestras'] <= 1400) & 
    (datos_cargados['Dedo'] == 'Indice')
]

# Obtén los valores de la columna 'Valores' en ese rango de muestras del dedo "Índice"
valores_rango_indice = muestras_rango_indice['Valores']

# Realiza la FFT de los valores en el rango seleccionado del dedo "Índice"
fft_valores_rango_indice = np.fft.fft(valores_rango_indice)

# Encuentra la magnitud de las amplitudes de la FFT del dedo "Índice"
magnitud_fft_indice = np.abs(fft_valores_rango_indice)

# Encuentra las frecuencias correspondientes para el dedo "Índice"
frecuencias_indice = np.fft.fftfreq(len(valores_rango_indice))
mitad_frecuencias_indice = frecuencias_indice[:len(frecuencias_indice)//2]
mitad_magnitudes_fft_indice = np.abs(fft_valores_rango_indice)[:len(magnitud_fft_indice)//2]



# Crear el gráfico de la FFT del dedo "Índice" con Plotly
fig_fft_indice = px.line(
    x=mitad_frecuencias_indice,
    y=mitad_magnitudes_fft_indice,
    template="plotly_dark",
    title="FFT del Dedo Índice",
)


# Crear el gráfico de la FFT del dedo "Anular" con Plotly
fig_fft_anular = px.line(
    x=mitad_frecuencias_anular,
    y=mitad_magnitudes_fft_anular,
    template="plotly_dark",
    title="FFT del Dedo Anular",
)


# Mostrar los gráficos interactivos utilizando Plotly
fig_fft_indice.show()
fig_fft_anular.show()


In [2]:
muestras_rango_anular_0 = datos_cargados[
    (datos_cargados['Muestras'] >= 320) & 
    (datos_cargados['Muestras'] <= 370) & 
    (datos_cargados['Dedo'] == 'Anular')
]['Valores']
muestras_rango_anular_1 = datos_cargados[
    (datos_cargados['Muestras'] >= 444) & 
    (datos_cargados['Muestras'] <= 500) & 
    (datos_cargados['Dedo'] == 'Anular')
]['Valores']
muestras_rango_anular_2 = datos_cargados[
    (datos_cargados['Muestras'] >= 560) & 
    (datos_cargados['Muestras'] <= 618) & 
    (datos_cargados['Dedo'] == 'Anular')
]['Valores']
muestras_rango_anular_3 = datos_cargados[
    (datos_cargados['Muestras'] >= 680) & 
    (datos_cargados['Muestras'] <= 740) & 
    (datos_cargados['Dedo'] == 'Anular')
]['Valores']
muestras_rango_anular_4 = datos_cargados[
    (datos_cargados['Muestras'] >= 800) & 
    (datos_cargados['Muestras'] <= 870) & 
    (datos_cargados['Dedo'] == 'Anular')
]['Valores']
muestras_rango_anular_5 = datos_cargados[
    (datos_cargados['Muestras'] >= 920) & 
    (datos_cargados['Muestras'] <= 990) & 
    (datos_cargados['Dedo'] == 'Anular')
]['Valores']
muestras_rango_anular_6 = datos_cargados[
    (datos_cargados['Muestras'] >= 1050) & 
    (datos_cargados['Muestras'] <= 1115) & 
    (datos_cargados['Dedo'] == 'Anular')
]['Valores']

muestras_rango_anular = [
    muestras_rango_anular_0,
    muestras_rango_anular_1,
    muestras_rango_anular_2,
    muestras_rango_anular_3,
    muestras_rango_anular_4,
    muestras_rango_anular_5,
    muestras_rango_anular_6
]

muestras_rango_indice_0 = datos_cargados[
    (datos_cargados['Muestras'] >= 190) & 
    (datos_cargados['Muestras'] <= 240) & 
    (datos_cargados['Dedo'] == 'Indice')
]['Valores']
muestras_rango_indice_1 = datos_cargados[
    (datos_cargados['Muestras'] >= 327) & 
    (datos_cargados['Muestras'] <= 380) & 
    (datos_cargados['Dedo'] == 'Indice')
]['Valores']
muestras_rango_indice_2 = datos_cargados[
    (datos_cargados['Muestras'] >= 460) & 
    (datos_cargados['Muestras'] <= 520) & 
    (datos_cargados['Dedo'] == 'Indice')
]['Valores']
muestras_rango_indice_3 = datos_cargados[
    (datos_cargados['Muestras'] >= 590) & 
    (datos_cargados['Muestras'] <= 650) & 
    (datos_cargados['Dedo'] == 'Indice')
]['Valores']
muestras_rango_indice_4 = datos_cargados[
    (datos_cargados['Muestras'] >= 740) & 
    (datos_cargados['Muestras'] <= 800) & 
    (datos_cargados['Dedo'] == 'Indice')
]['Valores']
muestras_rango_indice_5 = datos_cargados[
    (datos_cargados['Muestras'] >= 870) & 
    (datos_cargados['Muestras'] <= 930) & 
    (datos_cargados['Dedo'] == 'Indice')
]['Valores']
muestras_rango_indice_6 = datos_cargados[
    (datos_cargados['Muestras'] >= 1000) & 
    (datos_cargados['Muestras'] <= 1070) & 
    (datos_cargados['Dedo'] == 'Indice')
]['Valores']

muestras_rango_indice = [
    muestras_rango_indice_0,
    muestras_rango_indice_1,
    muestras_rango_indice_2,
    muestras_rango_indice_3,
    muestras_rango_indice_4,
    muestras_rango_indice_5,
    muestras_rango_indice_6
]

muestras_rango_medio_0 = datos_cargados[
    (datos_cargados['Muestras'] >= 190) & 
    (datos_cargados['Muestras'] <= 240) & 
    (datos_cargados['Dedo'] == 'Medio')
]['Valores']
muestras_rango_medio_1 = datos_cargados[
    (datos_cargados['Muestras'] >= 300) & 
    (datos_cargados['Muestras'] <= 360) & 
    (datos_cargados['Dedo'] == 'Medio')
]['Valores']
muestras_rango_medio_2 = datos_cargados[
    (datos_cargados['Muestras'] >= 430) & 
    (datos_cargados['Muestras'] <= 490) & 
    (datos_cargados['Dedo'] == 'Medio')
]['Valores']
muestras_rango_medio_3 = datos_cargados[
    (datos_cargados['Muestras'] >= 550) & 
    (datos_cargados['Muestras'] <= 600) & 
    (datos_cargados['Dedo'] == 'Medio')
]['Valores']
muestras_rango_medio_4 = datos_cargados[
    (datos_cargados['Muestras'] >= 670) & 
    (datos_cargados['Muestras'] <= 720) & 
    (datos_cargados['Dedo'] == 'Medio')
]['Valores']
muestras_rango_medio_5 = datos_cargados[
    (datos_cargados['Muestras'] >= 790) & 
    (datos_cargados['Muestras'] <= 835) & 
    (datos_cargados['Dedo'] == 'Medio')
]['Valores']
muestras_rango_medio_6 = datos_cargados[
    (datos_cargados['Muestras'] >= 900) & 
    (datos_cargados['Muestras'] <= 950) & 
    (datos_cargados['Dedo'] == 'Medio')
]['Valores']

muestras_rango_medio = [
    muestras_rango_medio_0,
    muestras_rango_medio_1,
    muestras_rango_medio_2,
    muestras_rango_medio_3,
    muestras_rango_medio_4,
    muestras_rango_medio_5,
    muestras_rango_medio_6
]



## Transformada Wavelet

### Funciones Wavelet

In [3]:
import pywt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# * "Wavelet de Coiflet (coif): Las wavelets de la familia Coiflet son conocidas
# * por su capacidad para manejar señales con comportamientos
# * irregulares o con rápidos cambios.
# * La Coiflet 4 o Coiflet 6 podrían ser opciones a considerar".

wavelet = 'coif4' 
nivel_descomposicion = 4  

def transformadaWavelet (muestras):
     
    coefs = pywt.wavedec(muestras, wavelet, level=nivel_descomposicion)
    return coefs

def reconstruccionWavelet(coefs):
    coefs_reconstruccion = coefs[:3] + [None] * 2
    senal_reconstruida = pywt.waverec(coefs_reconstruccion, wavelet)
    return senal_reconstruida



def graficar_coeficientes_de_muestra(coefs_anulares):
    fig = make_subplots(rows=nivel_descomposicion + 1, cols=1, subplot_titles=[f'Coeficiente {i+1}' for i in range(nivel_descomposicion + 1)])

    for i in range(nivel_descomposicion + 1):
        for j, coeficientes_muestra in enumerate(coefs_anulares):
            coef = coeficientes_muestra[i]
            fig.add_trace(go.Scatter(y=coef, mode='lines', name=f'Coef {i+1} Muestra {j}'), row=i+1, col=1)

        fig.update_xaxes(title_text='Muestras', row=i+1, col=1)
        fig.update_yaxes(title_text='Coeficiente', row=i+1, col=1)

    fig.update_layout(height=1000, width=1600, template="plotly_dark")
    fig.show()

def combinar_coefs_transpuestos(coefs_lista, nombre_dedo):
    dataframes = []
    
    for coefs in coefs_lista:
        df = pd.DataFrame(coefs).T
        df['Dedo'] = nombre_dedo
        dataframes.append(df)
    
    resultado = pd.concat(dataframes, ignore_index=True)
    return resultado

### Obtención de coeficientes Wavelet

In [5]:
coefs_anulares = []
coefs_indice = []
coefs_medio = []

for muestras_rango in muestras_rango_anular:
    coeficientes = transformadaWavelet(muestras_rango)
    coefs_anulares.append(coeficientes)


for muestras_rango in muestras_rango_indice:
    coeficientes = transformadaWavelet(muestras_rango)
    coefs_indice.append(coeficientes)

for muestras_rango in muestras_rango_medio:
    coeficientes = transformadaWavelet(muestras_rango)
    coefs_medio.append(coeficientes)



dataframe_anular = combinar_coefs_transpuestos(coefs_anulares,'Anular')
dataframe_indice = combinar_coefs_transpuestos(coefs_indice,'Indice')
dataframe_medio = combinar_coefs_transpuestos(coefs_medio,'Medio')
dataframe_lista = [dataframe_anular,dataframe_indice,dataframe_medio]


dataframe_completo = pd.concat(dataframe_lista ,ignore_index=True)


c:\Users\Isaia\AppData\Local\Programs\Python\Python311\Lib\site-packages\pywt\_multilevel.py:43: UserWarning: Level value of 4 is too high: all coefficients will experience boundary effects.
  warnings.warn(


### Gráficos Wavelet

In [ ]:

graficar_coeficientes_de_muestra(coefs_anulares)
graficar_coeficientes_de_muestra(coefs_indice)
graficar_coeficientes_de_muestra(coefs_medio)


### Reconstrucción Wavelet

In [ ]:

# senal_reconstruida_anular=reconstruccionWavelet(coefs_anulares[0])
# senal_reconstruida_indice=reconstruccionWavelet(coefs_indice)
senal_reconstruida_medio=reconstruccionWavelet(coefs_anulares[2])

basetime=list(range(100, 1001));
fig = px.line(template="plotly_dark")
fig.add_scatter(x=basetime,y=senal_reconstruida_medio, mode='lines', name='Medio 0')
fig.add_scatter(x=basetime,y=muestras_anular, mode='lines', name='Medio 0')
fig.add_scatter(x=basetime,y=muestras_rango_anular[2], mode='lines', name='Medio RAW 0')

# fig.add_scatter(y=senal_reconstruida_indice, mode='lines', name='Anular 1')

fig.show()